## This file does following:
1. Extract a data for a city from API
2. Store temporarily in a json file
3. Combine openweather and meteo data
4. Train a model and create prediction

In [1]:
try:
    import requests
except:
    !pip install requests
    import requests

try:
    import json
except:
    !pip install json
    import json

try:
    import pandas as pd
except:
    !pip install pandas
    import pandas as pd

try:
    import numpy as np
except:
    !pip install numpy
    import numpy as np

try:
    import os
except:
    !pip install os
    import os

# get current epoch time
import time
from datetime import datetime

# Define start and end dates
start_date = '2019-1-1'
end_date = '2025-06-30'

# Convert to datetime objects
dt_start = datetime.strptime(start_date, '%Y-%m-%d')
dt_end = datetime.strptime(end_date, '%Y-%m-%d')

# Convert to Unix epoch
epoch_start = int(dt_start.timestamp())
epoch_end = int(dt_end.timestamp())

print(f'Epoch Start: {epoch_start}')  # 1609459200
print(f'Epoch End: {epoch_end}')      # 1703980800

# convert epoch time to human readable format
import datetime
date_start = datetime.datetime.fromtimestamp(epoch_start).strftime('%Y-%m-%d')
date_end = datetime.datetime.fromtimestamp(epoch_end).strftime('%Y-%m-%d')
print(f'Date Start: {date_start}')
print(f'Date End: {date_end}')

api_key = 'f2bae9cda661e3f7f0cd2abf77705238'

Epoch Start: 1546282800
Epoch End: 1751223600
Date Start: 2019-01-01
Date End: 2025-06-30


In [2]:
# coordinates

coord = [[1, (43.0033629, 41.0192741)],
 [2, (34.5260109, 69.1776838)],
 [3, (34.67543085, 32.83482632147818)],
 [4, (41.3305141, 19.825562857582966)],
 [5, (36.7753606, 3.0601882)],
 [6, (-14.2754786, -170.7048298)],
 [7, (42.5069391, 1.5212467)],
 [8, (-8.8272699, 13.2439512)],
 [9, (51.48639195, 0.03616296186722791)],
 [10, (47.5614705, -52.7126162)],
 [11, (-34.6075682, -58.4370894)],
 [12, (40.1776245, 44.5126174)],
 [13, (12.5268736, -70.0356845)],
 [14, (6.8137426, -58.1624465)],
 [15, (-35.2975906, 149.1012676)],
 [16, (48.2083537, 16.3725042)],
 [17, (40.3755885, 49.8328009)],
 [18, (25.0782266, -77.3383438)],
 [19, (26.2235041, 50.5822436)],
 [20, (23.7644025, 90.389015)],
 [21, (13.0977832, -59.6184184)],
 [22, (53.9024716, 27.5618225)],
 [23, (50.8465573, 4.351697)],
 [24, (17.250199, -88.770018)],
 [25, (6.4990718, 2.6253361)],
 [26, (43.2560802, -79.8728583)],
 [27, (27.4713546, 89.6336729)],
 [28, (-19.0477251, -65.2594306)],
 [29, (-16.4955455, -68.1336229)],
 [30, (43.8519774, 18.3866868)],
 [31, (-24.6581357, 25.9088474)],
 [32, (-10.3333333, -53.2)],
 [33, (18.4257128, -64.6232227)],
 [34, (4.8895453, 114.9417574)],
 [35, (-15.2538402, 48.2562163)],
 [36, (12.3681873, -1.5270944)],
 [37, (-3.3638125, 29.3675028)],
 [38, (11.568271, 104.9224426)],
 [39, (3.8689867, 11.5213344)],
 [40, (45.4208777, -75.6901106)],
 [41, (14.9162811, -23.5095095)],
 [42, (5.4141619, 100.3287352)],
 [43, (4.3907153, 18.5509126)],
 [44, (12.1191543, 15.0502758)],
 [45, (9.8694792, -83.7980749)],
 [46, (39.906217, 116.3912757)],
 [47, (-10.4213344, 105.674413)],
 [48, (-12.145866, 96.84227540282865)],
 [49, (4.6534649, -74.0836453)],
 [50, (-11.6931255, 43.2543044)],
 [51, (-21.2074736, -159.7708145)],
 [52, (9.9325427, -84.0795782)],
 [53, (45.84264135, 15.962231476593626)],
 [54, (23.135305, -82.3589631)],
 [55, (12.1066695, -68.9351307)],
 [56, (35.1748976, 33.3638568)],
 [57, (50.0874654, 14.4212535)],
 [58, (6.809107, -5.273263)],
 [59, (-4.3217055, 15.3125974)],
 [60, (55.6867243, 12.5700724)],
 [61, (11.8145966, 42.8453061)],
 [62, (48.7710371, -95.7697882)],
 [63, (18.4801972, -69.942111)],
 [64, (28.6517178, 77.2219388)],
 [65, (-27.1481172, -109.427344)],
 [66, (-0.2201641, -78.5123274)],
 [67, (30.0443879, 31.2357257)],
 [68, (13.6989939, -89.1914249)],
 [69, (3.752828, 8.780061)],
 [70, (15.3389667, 38.9326763)],
 [71, (59.4372155, 24.7453688)],
 [72, (9.0107934, 38.7612525)],
 [73, (-51.6950575, -57.8491693)],
 [74, (62.012, -6.768)],
 [75, (6.920744, 158.1627143)],
 [76, (-18.1415884, 178.4421662)],
 [77, (60.1674881, 24.9427473)],
 [78, (48.8588897, 2.3200410217200766)],
 [79, (4.9371143, -52.3258307)],
 [80, (-17.5373835, -149.5659964)],
 [81, (0.390002, 9.454001)],
 [82, (13.4410165, -16.56275092072591)],
 [83, (41.6934591, 44.8014495)],
 [84, (52.5170365, 13.3888599)],
 [85, (5.5571096, -0.2012376)],
 [86, (36.1285933, -5.3474761)],
 [87, (37.9839412, 23.7283052)],
 [88, (64.175029, -51.7355386)],
 [89, (12.0535331, -61.751805)],
 [90, (13.4727446, 144.75201822544284)],
 [91, (14.6222328, -90.5185188)],
 [92, (49.4568142, -2.5389979)],
 [93, (9.5170602, -13.6998434)],
 [94, (11.861324, -15.583055)],
 [95, (6.8137426, -58.1624465)],
 [96, (18.547327, -72.3395928)],
 [97, (14.1057433, -87.2040052)],
 [98, (47.4979937, 19.0403594)],
 [99, (64.145981, -21.9422367)],
 [100, (28.6138954, 77.2090057)],
 [101, (-6.1753942, 106.827183)],
 [102, (35.6892523, 51.3896004)],
 [103, (33.3061701, 44.3872213)],
 [104, (53.3498006, -6.2602964)],
 [105, (39.7628415, -88.2170516)],
 [106, (31.79592425, 35.21198075969497)],
 [107, (41.8933203, 12.4829321)],
 [108, (17.9712148, -76.7928128)],
 [109, (35.6828387, 139.7594549)],
 [110, (49.1856637, -2.1102277)],
 [111, (31.9515694, 35.9239625)],
 [112, (51.1282205, 71.4306682)],
 [113, (-1.2832533, 36.8172449)],
 [114, (1.4845541999999998, 172.96896482000938)],
 [115, (42.6638771, 21.1640849)],
 [116, (29.3796532, 47.9734174)],
 [117, (42.8765615, 74.6070079)],
 [118, (17.9640988, 102.6133707)],
 [119, (56.9493977, 24.1051846)],
 [120, (33.8959203, 35.47843)],
 [121, (-29.310054, 27.478222)],
 [122, (6.328034, -10.797788)],
 [123, (32.896672, 13.1777923)],
 [124, (47.1392862, 9.5227962)],
 [125, (54.6870458, 25.2829111)],
 [126, (49.8158683, 6.1296751)],
 [127, (41.9960924, 21.4316495)],
 [128, (-18.9100122, 47.5255809)],
 [129, (-13.9875107, 33.768144)],
 [130, (3.1516964, 101.6942371)],
 [131, (4.1779879, 73.5107387)],
 [132, (12.61326555, -7.984739136241295)],
 [133, (35.8989818, 14.5136759)],
 [134, (7.0909924, 171.3816354)],
 [135, (18.061891799999998, -15.931148157832908)],
 [136, (-20.1624522, 57.5028044)],
 [137, (19.4326296, -99.1331785)],
 [138, (47.0245117, 28.8322923)],
 [139, (43.7323492, 7.4276832)],
 [140, (47.9184676, 106.9177016)],
 [141, (42.4415238, 19.2621081)],
 [142, (50.3712659, -4.1425658)],
 [143, (34.022405, -6.834543)],
 [144, (-25.966213, 32.56745)],
 [145, (19.7540045, 96.1344976)],
 [146, (39.8182044, 46.7517171)],
 [147, (-22.5776104, 17.0772739)],
 [148, (-0.5469030500000001, 166.92235277906863)],
 [149, (27.708317, 85.3205817)],
 [150, (52.3727598, 4.8936041)],
 [151, (-22.2745264, 166.442419)],
 [152, (-41.2887953, 174.7772114)],
 [153, (12.1544035, -86.2737642)],
 [154, (13.524834, 2.109823)],
 [155, (9.0643305, 7.4892974)],
 [156, (-19.0534159, -169.919199)],
 [157, (17.9712148, -76.7928128)],
 [158, (39.0167979, 125.7473609)],
 [159, (35.1748976, 33.3638568)],
 [160, (54.596391, -5.9301829)],
 [161, (15.1909825, 145.746743003024)],
 [162, (59.9133301, 10.7389701)],
 [163, (23.61515, 58.5912467)],
 [164, (33.6938118, 73.0651511)],
 [165, (7.5006446, 134.6242864)],
 [166, (31.79592425, 35.21198075969497)],
 [167, (30.1600827, -85.6545729)],
 [168, (-9.4743301, 147.1599504)],
 [169, (-25.2800459, -57.6343814)],
 [170, (-12.0621065, -77.0365256)],
 [171, (14.5948914, 120.9782618)],
 [172, (-25.066667, -130.100205)],
 [173, (52.2337172, 21.071432235636493)],
 [174, (38.7077507, -9.1365919)],
 [175, (18.465299, -66.116666)],
 [176, (25.2856329, 51.5264162)],
 [177, (25.0375198, 121.5636796)],
 [178, (-4.2694407, 15.2712256)],
 [179, (44.4361414, 26.1027202)],
 [180, (55.7504461, 37.6174943)],
 [181, (-1.950851, 30.061507)],
 [182, (17.8957043, -62.8508372)],
 [183, (37.2104434, -76.7738928)],
 [184, (17.2960919, -62.722301)],
 [185, (43.6779101, 3.9868904)],
 [186, (18.0668544, -63.0848869)],
 [187, (48.3832725, 7.4718731)],
 [188, (13.1561864, -61.2279621)],
 [189, (-13.8343691, -171.7692793)],
 [190, (43.9458623, 12.458306)],
 [191, (24.638916, 46.7160104)],
 [192, (55.9533456, -3.1883749)],
 [193, (14.693425, -17.447938)],
 [194, (44.8178131, 20.4568974)],
 [195, (-36.5986096, 144.6780052)],
 [196, (8.479004, -13.26795)],
 [197, (1.357107, 103.8194992)],
 [198, (18.0250713, -63.0483073)],
 [199, (48.1516988, 17.1093063)],
 [200, (46.0500268, 14.5069289)],
 [201, (-9.437797549999999, 159.9624174786047)],
 [202, (2.0349312, 45.3419183)],
 [203, (9.561733, 44.061803)],
 [204, (-25.7459277, 28.1879101)],
 [205, (-54.2806481, -36.5098)],
 [206, (37.5666791, 126.9782914)],
 [207, (42.2251222, 43.9708477)],
 [208, (4.8459246, 31.5959173)],
 [209, (40.4167047, -3.7035825)],
 [210, (6.8883219, 79.9187415)],
 [211, (15.5635972, 32.5349123)],
 [212, (5.8247628, -55.1703941)],
 [213, (-26.325745, 31.144663)],
 [214, (59.3251172, 18.0710935)],
 [215, (46.9482713, 7.4514512)],
 [216, (33.5130695, 36.3095814)],
 [217, (0.3389242, 6.7313031)],
 [218, (38.5425835, 68.81521423589763)],
 [219, (-6.1791181, 35.7468174)],
 [220, (13.7524938, 100.4935089)],
 [221, (6.130419, 1.215829)],
 [222, (-21.1343401, -175.201808)],
 [223, (46.8371103, 29.6109989)],
 [224, (10.6572678, -61.5180173)],
 [225, (-37.0675894, -12.3108303)],
 [226, (33.8439408, 9.400138)],
 [227, (39.9207886, 32.8540482)],
 [228, (37.9404648, 58.3823487)],
 [229, (21.4607723, -71.1399956)],
 [230, (-8.5199633, 179.1982548)],
 [231, (0.3177137, 32.5813539)],
 [232, (50.4500336, 30.5241361)],
 [233, (24.4538352, 54.3774014)],
 [234, (51.5073219, -0.1276474)],
 [235, (38.9074322, -77.0350922)],
 [236, (18.341137, -64.932789)],
 [237, (-34.9058916, -56.1913095)],
 [238, (41.3123363, 69.2787079)],
 [239, (-17.7414972, 168.3150163)],
 [240, (41.903411, 12.4528527)],
 [241, (10.4813175, -66.8796057)],
 [242, (21.0294498, 105.8544441)],
 [243, (51.4816546, -3.1791934)],
 [244, (-13.2820419, -176.1740224)],
 [245, (27.154512, -13.1953921)],
 [246, (15.3538569, 44.2058841)],
 [247, (-15.4164124, 28.2824793)],
 [248, (-17.831773, 31.045686)]]

In [3]:
# cities
cities = [[1, 'Sukhumi'],
 [2, 'Kabul'],
 [3, 'Episkopi Cantonment'],
 [4, 'Tirana'],
 [5, 'Algiers'],
 [6, 'Pago Pago'],
 [7, 'Andorra la Vella'],
 [8, 'Luanda'],
 [9, 'The Valley'],
 [10, "St. John's"],
 [11, 'Buenos Aires'],
 [12, 'Yerevan'],
 [13, 'Oranjestad'],
 [14, 'Georgetown'],
 [15, 'Canberra'],
 [16, 'Vienna'],
 [17, 'Baku'],
 [18, 'Nassau'],
 [19, 'Manama'],
 [20, 'Dhaka'],
 [21, 'Bridgetown'],
 [22, 'Minsk'],
 [23, 'Brussels'],
 [24, 'Belmopan'],
 [25, 'Porto-Novo'],
 [26, 'Hamilton'],
 [27, 'Thimphu'],
 [28, 'Sucre'],
 [29, 'La Paz'],
 [30, 'Sarajevo'],
 [31, 'Gaborone'],
 [32, 'Brasília'],
 [33, 'Road Town'],
 [34, 'Bandar Seri Begawan'],
 [35, 'Sofia'],
 [36, 'Ouagadougou'],
 [37, 'Bujumbura'],
 [38, 'Phnom Penh'],
 [39, 'Yaoundé'],
 [40, 'Ottawa'],
 [41, 'Praia'],
 [42, 'George Town'],
 [43, 'Bangui'],
 [44, "N'Djamena"],
 [45, 'Santiago'],
 [46, 'Beijing'],
 [47, 'Flying Fish Cove'],
 [48, 'West Island'],
 [49, 'Bogotá'],
 [50, 'Moroni'],
 [51, 'Avarua'],
 [52, 'San José'],
 [53, 'Zagreb'],
 [54, 'Havana'],
 [55, 'Willemstad'],
 [56, 'Nicosia'],
 [57, 'Prague'],
 [58, 'Yamoussoukro'],
 [59, 'Kinshasa'],
 [60, 'Copenhagen'],
 [61, 'Djibouti'],
 [62, 'Roseau'],
 [63, 'Santo Domingo'],
 [64, 'Dili'],
 [65, 'Hanga Roa'],
 [66, 'Quito'],
 [67, 'Cairo'],
 [68, 'San Salvador'],
 [69, 'Malabo'],
 [70, 'Asmara'],
 [71, 'Tallinn'],
 [72, 'Addis Ababa'],
 [73, 'Stanley'],
 [74, 'Tórshavn'],
 [75, 'Palikir'],
 [76, 'Suva'],
 [77, 'Helsinki'],
 [78, 'Paris'],
 [79, 'Cayenne'],
 [80, 'Papeete'],
 [81, 'Libreville'],
 [82, 'Banjul'],
 [83, 'Tbilisi'],
 [84, 'Berlin'],
 [85, 'Accra'],
 [86, 'Gibraltar'],
 [87, 'Athens'],
 [88, 'Nuuk'],
 [89, "St. George's"],
 [90, 'Hagåtña'],
 [91, 'Guatemala City'],
 [92, 'St. Peter Port'],
 [93, 'Conakry'],
 [94, 'Bissau'],
 [95, 'Georgetown'],
 [96, 'Port-au-Prince'],
 [97, 'Tegucigalpa'],
 [98, 'Budapest'],
 [99, 'Reykjavík'],
 [100, 'New Delhi'],
 [101, 'Jakarta'],
 [102, 'Tehran'],
 [103, 'Baghdad'],
 [104, 'Dublin'],
 [105, 'Douglas'],
 [106, 'Jerusalem'],
 [107, 'Rome'],
 [108, 'Kingston'],
 [109, 'Tokyo'],
 [110, 'St. Helier'],
 [111, 'Amman'],
 [112, 'Astana'],
 [113, 'Nairobi'],
 [114, 'Tarawa'],
 [115, 'Pristina'],
 [116, 'Kuwait City'],
 [117, 'Bishkek'],
 [118, 'Vientiane'],
 [119, 'Riga'],
 [120, 'Beirut'],
 [121, 'Maseru'],
 [122, 'Monrovia'],
 [123, 'Tripoli'],
 [124, 'Vaduz'],
 [125, 'Vilnius'],
 [126, 'Luxembourg'],
 [127, 'Skopje'],
 [128, 'Antananarivo'],
 [129, 'Lilongwe'],
 [130, 'Kuala Lumpur'],
 [131, 'Malé'],
 [132, 'Bamako'],
 [133, 'Valletta'],
 [134, 'Majuro'],
 [135, 'Nouakchott'],
 [136, 'Port Louis'],
 [137, 'Mexico City'],
 [138, 'Chisinau'],
 [139, 'Monaco'],
 [140, 'Ulaanbaatar'],
 [141, 'Podgorica'],
 [142, 'Plymouth'],
 [143, 'Rabat'],
 [144, 'Maputo'],
 [145, 'Naypyidaw'],
 [146, 'Stepanakert'],
 [147, 'Windhoek'],
 [148, 'Yaren'],
 [149, 'Kathmandu'],
 [150, 'Amsterdam'],
 [151, 'Nouméa'],
 [152, 'Wellington'],
 [153, 'Managua'],
 [154, 'Niamey'],
 [155, 'Abuja'],
 [156, 'Alofi'],
 [157, 'Kingston'],
 [158, 'Pyongyang'],
 [159, 'Nicosia'],
 [160, 'Belfast'],
 [161, 'Saipan'],
 [162, 'Oslo'],
 [163, 'Muscat'],
 [164, 'Islamabad'],
 [165, 'Ngerulmud'],
 [166, 'Jerusalem'],
 [167, 'Panama City'],
 [168, 'Port Moresby'],
 [169, 'Asunción'],
 [170, 'Lima'],
 [171, 'Manila'],
 [172, 'Adamstown'],
 [173, 'Warsaw'],
 [174, 'Lisbon'],
 [175, 'San Juan'],
 [176, 'Doha'],
 [177, 'Taipei'],
 [178, 'Brazzaville'],
 [179, 'Bucharest'],
 [180, 'Moscow'],
 [181, 'Kigali'],
 [182, 'Gustavia'],
 [183, 'Jamestown'],
 [184, 'Basseterre'],
 [185, 'Castries'],
 [186, 'Marigot'],
 [187, 'St. Pierre'],
 [188, 'Kingstown'],
 [189, 'Apia'],
 [190, 'San Marino'],
 [191, 'Riyadh'],
 [192, 'Edinburgh'],
 [193, 'Dakar'],
 [194, 'Belgrade'],
 [195, 'Victoria'],
 [196, 'Freetown'],
 [197, 'Singapore'],
 [198, 'Philipsburg'],
 [199, 'Bratislava'],
 [200, 'Ljubljana'],
 [201, 'Honiara'],
 [202, 'Mogadishu'],
 [203, 'Hargeisa'],
 [204, 'Pretoria'],
 [205, 'Grytviken'],
 [206, 'Seoul'],
 [207, 'Tskhinvali'],
 [208, 'Juba'],
 [209, 'Madrid'],
 [210, 'Sri Jayawardenapura Kotte'],
 [211, 'Khartoum'],
 [212, 'Paramaribo'],
 [213, 'Mbabane'],
 [214, 'Stockholm'],
 [215, 'Bern'],
 [216, 'Damascus'],
 [217, 'São Tomé'],
 [218, 'Dushanbe'],
 [219, 'Dodoma'],
 [220, 'Bangkok'],
 [221, 'Lomé'],
 [222, 'Nukuʻalofa'],
 [223, 'Tiraspol'],
 [224, 'Port of Spain'],
 [225, 'Edinburgh of the Seven Seas'],
 [226, 'Tunis'],
 [227, 'Ankara'],
 [228, 'Ashgabat'],
 [229, 'Cockburn Town'],
 [230, 'Funafuti'],
 [231, 'Kampala'],
 [232, 'Kiev'],
 [233, 'Abu Dhabi'],
 [234, 'London'],
 [235, 'Washington, D.C.'],
 [236, 'Charlotte Amalie'],
 [237, 'Montevideo'],
 [238, 'Tashkent'],
 [239, 'Port Vila'],
 [240, 'Vatican City'],
 [241, 'Caracas'],
 [242, 'Hanoi'],
 [243, 'Cardiff'],
 [244, 'Mata-Utu'],
 [245, 'El Aaiún'],
 [246, 'Sanaá'],
 [247, 'Lusaka'],
 [248, 'Harare']]

In [4]:
# Defining starting and ending index for the loop
start_index = 0
end_index = len(coord)
print(f'Start index: {start_index}')
print(f'End index: {end_index}')

Start index: 0
End index: 248


In [5]:
# this cell combines two json files into one dataframe after all the preprocessing steps

# fucntion for date conversion of openweather data
def convert_date(ts):

    ts=pd.to_datetime(ts, unit='s')
    year=ts.year
    month=ts.month
    day=ts.day
    hour=ts.hour
    return [year, month, day, hour]

# combining the two json files into one dataframe for a particular zone
def combine_data(zone,city):

        # reading openweather data
        with open(f"temp_openweather.json") as f:
                openweather = json.load(f)

        # getting longitude and latitude
        longitude = openweather['coord']['lon']
        latitude = openweather['coord']['lat']

        # converting openweather data to dataframe
        arr=[convert_date(openweather['list'][i]['dt'])+[zone, city, longitude,latitude,openweather['list'][0]['main']['aqi']]+
                list(openweather['list'][i]['components'].values()) for i in range(len(openweather['list']))]
        openweather = pd.DataFrame(arr, columns=['year','month','day','hour','zone', 'city',
                'longitude','latitude','aqi','co','no','no2','o3','so2','pm2_5','pm10','nh3'])


        # reading meteo data
        with open(f"temp_meteo.json") as f:
                meteo = json.load(f)
        
        arr=[convert_date(pd.to_datetime(meteo['hourly']['time'][i]))+[longitude,latitude,
                meteo['hourly']['temperature_2m'][i], meteo['hourly']['dewpoint_2m'][i]] for i in range(len(meteo['hourly']['time']))]
        meteo = pd.DataFrame(arr, columns=['year','month','day','hour','longitude','latitude','temperature','dewpt'])

        # merging openweather and weatherbit data
        # print(f"merging openweather and meteo data for zone {zone}")
        return pd.merge(openweather, meteo, on=['year','month','day','hour','longitude','latitude'])#.drop([8734])

In [6]:
def get_data(i):
    # get data and save to JSON file

    url = f"https://api.openweathermap.org/data/2.5/air_pollution/history?lat={coord[i][1][0]}&lon={coord[i][1][1]}&start={epoch_start}&end={epoch_end}&appid={api_key}"
    response = requests.get(url)
    with open('temp_openweather.json', 'w') as f:
        json.dump(response.json(), f, indent=4)

    url = f"https://archive-api.open-meteo.com/v1/era5?latitude={coord[i][1][0]}&longitude={coord[i][1][1]}&start_date={date_start}&end_date={date_end}&hourly=temperature_2m,dewpoint_2m"
    response = requests.get(url)
    with open('temp_meteo.json', 'w') as f:
        json.dump(response.json(), f, indent=4)

    df = combine_data(i+1, cities[i][1])
    df['date'] = pd.to_datetime(df[['year', 'month', 'day', 'hour']]).dt.strftime('%Y-%m-%d %H:%M:%S')
    df['date'] = pd.to_datetime(df['date']).dt.strftime('%B %d, %Y, %I:%M %p')
    df.dropna(inplace=True)

    # immediately append to CSV after processing each zone
    df.to_csv('beijing.csv', mode='a', header=not os.path.exists('beijing.csv'), index=False)

    os.remove('temp_openweather.json')
    os.remove('temp_meteo.json')

    print(f'Zone {i+1} done')

    return df


In [7]:
df=get_data(45)
#df.to_csv('beijing.csv', mode='a', header=False, index=False)

Zone 46 done


In [ ]:
# for i in range(0,25):
#     df=get_data(i)
#     df.to_csv('1.csv', mode='a', header=False, index=False)

In [ ]:
# for i in range(25,50):
#     df=get_data(i)
#     df.to_csv('2.csv', mode='a', header=False, index=False)

In [ ]:
# for i in range(50,75):
#     df=get_data(i)
#     df.to_csv('3.csv', mode='a', header=False, index=False)

In [ ]:
# for i in range(75,100):
#     df=get_data(i)
#     df.to_csv('4.csv', mode='a', header=False, index=False)

In [ ]:
# for i in range(100,125):
#     df=get_data(i)
#     df.to_csv('5.csv', mode='a', header=False, index=False)

In [ ]:
# for i in range(125,150):
#     df=get_data(i)
#     df.to_csv('6.csv', mode='a', header=False, index=False)

In [ ]:
# for i in range(150,175):
#     df=get_data(i)
#     df.to_csv('7.csv', mode='a', header=False, index=False)

In [ ]:
# for i in range(175,200):
#     df=get_data(i)
#     df.to_csv('8.csv', mode='a', header=False, index=False)

In [ ]:
# for i in range(200,225):
#     df=get_data(i)
#     df.to_csv('9.csv', mode='a', header=False, index=False)

In [ ]:
# for i in range(225,248):
#     df=get_data(i)
#     df.to_csv('10.csv', mode='a', header=False, index=False)

## Compiling the Raw Data File

In [6]:
# define header
header = ['year','month','day','hour','zone','city','longitude','latitude','aqi','co','no','no2','o3','so2','pm2_5','pm10','nh3','temperature','dewpt','date','smog']

In [7]:
df = pd.read_csv('Batch Data/1.csv', names=header)
df = df[['date','zone','longitude','latitude','temperature','pm10','pm2_5','smog']]
df.to_csv('raw_data.csv', index=False)

In [8]:
for i in range(2, 11):
    df = pd.read_csv('Batch Data/'+str(i)+'.csv', names=header)
    df = df[['date','zone','longitude','latitude','temperature','pm10','pm2_5','smog']]
    df.to_csv('raw_data.csv', mode='a', header=False, index=False)
    print(f"{i} done")

2 done
3 done
4 done
5 done
6 done
7 done
8 done
9 done
10 done


In [ ]:
https://api.openweathermap.org/data/2.5/air_pollution/history?lat=43.0033629&lon=41.0192741&start=1609459200&end=1609462800&appid=f2bae9cda661e3f7f0cd2abf77705238

In [ ]:
https://archive-api.open-meteo.com/v1/era5?latitude=43.0033629&longitude=41.0192741&start_date=2021-01-01&end_date=2021-01-02&hourly=temperature_2m,dewpoint_2m